In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
df=pd.DataFrame(data=data.data,columns=data.feature_names)
df_target=data.target
df=pd.DataFrame(data=data.data,columns=data.feature_names)
df_target=data.target
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,radius error,texture error,perimeter error,area error,smoothness error,compactness error,concavity error,concave points error,symmetry error,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,1.0950,0.9053,8.589,153.40,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.01860,0.01340,0.01389,0.003532,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.006150,0.04006,0.03832,0.02058,0.02250,0.004571,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,0.4956,1.1560,3.445,27.23,0.009110,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.011490,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df,df_target, test_size=0.2, random_state=42)
# from sklearn.ensemble import RandomForestClassifier
# clf = RandomForestClassifier(n_estimators=100, random_state=0)
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier(n_neighbors=5)

In [ ]:
class silver_bhaediya:

  def __init__(self,objective_function,n_iteration=50,population_size=50):
    self.population_size = population_size
    self.n_iteration=n_iteration
    self.objective_function=objective_function

  def initialize_population(self,X):
        self.individuals =  np.random.randint(0,2,size=(self.population_size,X.shape[1]))


  def evaluate_fitness(self,model,X_train,y_train,X_valid,y_valid):
        scores =  []
        for i,individual in enumerate(self.individuals):
            chosen_features = [index for index in range(X_train.shape[1]) if individual[index]==1]
            X_train_copy = X_train.iloc[:,chosen_features]
            X_valid_copy = X_valid.iloc[:,chosen_features]
            score = self.objective_function(model,X_train_copy,y_train,X_valid_copy,y_valid)
            if score< self.best_score:
              self.best_score=score
              self.best_score_dimension=individual

            scores.append(score)
        self.fitness_scores = scores
 

  def sigmoid(self,x):
    return 1/(1+np.exp( -10*(x-0.5) ))
        
  def fit(self,model,X_train,y_train,X_valid,y_valid,method=2):
    self.initialize_population(X_train)
    self.best_score=np.inf
    self.best_score_dimension=np.ones(X_train.shape[1]) 

    self.alpha_wolf_dimension,self.alpha_wolf_fitness=np.ones(X_train.shape[1]),np.inf
    self.beta_wolf_dimension,self.beta_wolf_fitness=np.ones(X_train.shape[1]),np.inf
    self.delta_wolf_dimension,self.delta_wolf_fitness=np.ones(X_train.shape[1]),np.inf


    for i in range(self.n_iteration):
      a=2-2*((i+1)/self.n_iteration)
      self.evaluate_fitness(model,X_train,y_train,X_valid,y_valid)

      top_three_fitness_indexes=np.argsort(self.fitness_scores)[:3]

      for fit,dim in zip( np.array(self.fitness_scores)[top_three_fitness_indexes],self.individuals[top_three_fitness_indexes] ):
        
        if fit<self.alpha_wolf_fitness:
          self.delta_wolf_fitness=self.beta_wolf_fitness
          self.beta_wolf_fitness=self.alpha_wolf_fitness
          self.alpha_wolf_fitness=fit

          self.delta_wolf_dimension=self.beta_wolf_dimension
          self.beta_wolf_dimension=self.alpha_wolf_dimension
          self.alpha_wolf_dimension=dim
          continue

        if ((fit>self.alpha_wolf_fitness)&(fit<self.beta_wolf_fitness)):
          self.delta_wolf_fitness=self.beta_wolf_fitness
          self.beta_wolf_fitness=fit

          self.delta_wolf_dimension=self.beta_wolf_dimension
          self.beta_wolf_dimension=dim
          continue

        if ((fit>self.beta_wolf_fitness)&(fit<self.delta_wolf_fitness)):
          self.delta_wolf_fitness=fit
          self.delta_wolf_dimension=dim
          continue

      if (method==1)|(method==2):
        C1=2*np.random.random((self.population_size,X_train.shape[1]))
        A1=2*a*np.random.random((self.population_size,X_train.shape[1]))-a
        D_alpha=abs( C1*self.alpha_wolf_dimension - self.individuals )

        C2=2*np.random.random((self.population_size,X_train.shape[1]))
        A2=2*a*np.random.random((self.population_size,X_train.shape[1]))-a
        D_beta=abs( C2*self.beta_wolf_dimension - self.individuals )

        C3=2*np.random.random((self.population_size,X_train.shape[1]))
        A3=2*a*np.random.random((self.population_size,X_train.shape[1]))-a
        D_delta=abs( C3*self.delta_wolf_dimension - self.individuals )

      if method==2:
        X1=abs( self.alpha_wolf_dimension -A1*D_alpha )
        X2=abs( self.beta_wolf_dimension - A2*D_beta  )
        X3=abs( self.delta_wolf_dimension -A3*D_delta )
        self.individuals=np.where(np.random.uniform(size=(self.population_size,X_train.shape[1]))<= self.sigmoid((X1+X2+X3)/3),1,0)

      if method==1:
        Y1=np.where(  (self.alpha_wolf_dimension+ np.where(self.sigmoid(A1*D_alpha)>np.random.uniform(size=(self.population_size,X_train.shape[1])),1,0) ) >=1,1,0)
        Y2=np.where(  (self.beta_wolf_dimension+ np.where(self.sigmoid(A1*D_beta)>np.random.uniform(size=(self.population_size,X_train.shape[1])),1,0) ) >=1,1,0)
        Y3=np.where(  (self.delta_wolf_dimension+ np.where(self.sigmoid(A1*D_delta)>np.random.uniform(size=(self.population_size,X_train.shape[1])),1,0) ) >=1,1,0)
        r=np.random.uniform(size=(self.population_size,X_train.shape[1]))
        self.individuals[r<(1/3)]=Y1[r<(1/3)]
        self.individuals[(r>=(1/3))&(r<(2/3))]=Y2[(r>=(1/3))&(r<(2/3))]
        self.individuals[r>=(2/3)]=Y3[r>=(2/3)]
##############################################################################################################################################################
                                                            #COMPETITIVE
#############################################################################################################################################################
      if method=='competitive':
        indx=np.arange(0,self.population_size)
        np.random.shuffle(indx)
        grp=indx.reshape(int(self.population_size/2),2)
        R_value=0.9-0.9*((i+1)/self.n_iteration)

        winner_population=grp[np.arange(int(self.population_size/2)),np.array(self.fitness_scores)[grp].argmin(axis=1)]
        looser_population=grp[np.arange(int(self.population_size/2)),np.array(self.fitness_scores)[grp].argmax(axis=1)]

        C1=2*np.random.random((int(self.population_size/2),X_train.shape[1]))
        A1=2*a*np.random.random((int(self.population_size/2),X_train.shape[1]))-a
        D_alpha=abs( C1*self.alpha_wolf_dimension - (self.individuals[winner_population]-self.individuals[looser_population]) )  
        X1=abs( self.alpha_wolf_dimension -A1*D_alpha )

        C2=2*np.random.random((int(self.population_size/2),X_train.shape[1]))
        A2=2*a*np.random.random((int(self.population_size/2),X_train.shape[1]))-a
        D_beta=abs( C2*self.beta_wolf_dimension - (self.individuals[winner_population]-self.individuals[looser_population]) )  
        X2=abs( self.beta_wolf_dimension -A2*D_beta )

        C3=2*np.random.random((int(self.population_size/2),X_train.shape[1]))
        A3=2*a*np.random.random((int(self.population_size/2),X_train.shape[1]))-a
        D_delta=abs( C2*self.delta_wolf_dimension - (self.individuals[winner_population]-self.individuals[looser_population]) )  
        X3=abs( self.delta_wolf_dimension -A3*D_delta )

        self.individuals[looser_population]=np.where(np.random.uniform(size=(int(self.population_size/2),X_train.shape[1]))<= self.sigmoid((X1+X2+X3)/3),1,0)
        
        self.alpha_wolf_dimension=np.where( R_value>=np.random.uniform(size=(X_train.shape[1])),
                                           np.random.randint(0,2,size=(X_train.shape[1])), self.alpha_wolf_dimension )
        self.beta_wolf_dimension=np.where( R_value>=np.random.uniform(size=(X_train.shape[1])),
                                           np.random.randint(0,2,size=(X_train.shape[1])), self.beta_wolf_dimension )
        self.delta_wolf_dimension=np.where( R_value>=np.random.uniform(size=(X_train.shape[1])),
                                           np.random.randint(0,2,size=(X_train.shape[1])), self.delta_wolf_dimension )
        








      print('iteration ',i,' ',np.array(self.fitness_scores).min(),' |mean ',np.array(self.fitness_scores).mean(),' |best_score ',self.best_score)

    return self.best_score_dimension

    


In [ ]:
import scipy
from sklearn.metrics import log_loss

def score_model(model,X_train, y_train, X_valid, y_valid):
    model.fit(X_train,y_train)
    # return log_loss(y_valid,model.predict_proba(X_valid))
    P = (model.predict(X_valid) != y_valid).mean()
    alpha=0.01
    # Compute for the objective function
    j = alpha*(X_valid.shape[1]/30)+(1-alpha)*P
    return j
gaf=silver_bhaediya(score_model,50,20)
res=gaf.fit(clf,X_train, y_train,X_test, y_test,'competitive')

iteration  0   0.03071929824561403  |mean  0.05398245614035087  |best_score  0.03071929824561403
iteration  1   0.03071929824561403  |mean  0.05361315789473684  |best_score  0.03071929824561403
iteration  2   0.03071929824561403  |mean  0.055615789473684195  |best_score  0.03071929824561403
iteration  3   0.03071929824561403  |mean  0.05573245614035087  |best_score  0.03071929824561403
iteration  4   0.03071929824561403  |mean  0.06319736842105263  |best_score  0.03071929824561403
iteration  5   0.03071929824561403  |mean  0.05920614035087718  |best_score  0.03071929824561403
iteration  6   0.03071929824561403  |mean  0.06496842105263158  |best_score  0.03071929824561403
iteration  7   0.03071929824561403  |mean  0.05725263157894737  |best_score  0.03071929824561403
iteration  8   0.03071929824561403  |mean  0.052224561403508764  |best_score  0.03071929824561403
iteration  9   0.03071929824561403  |mean  0.053059649122807015  |best_score  0.03071929824561403
iteration  10   0.030719298

In [ ]:
np.arange(5)

array([0, 1, 2, 3, 4])

In [ ]:
import scipy
from sklearn.metrics import log_loss

def score_model(model,X_train, y_train, X_valid, y_valid):
    model.fit(X_train,y_train)
    # return log_loss(y_valid,model.predict_proba(X_valid))
    P = (model.predict(X_valid) != y_valid).mean()
    alpha=0.01
    # Compute for the objective function
    j = alpha*(X_valid.shape[1]/30)+(1-alpha)*P
    return j
gaf=silver_bhaediya(score_model,50,20)
res=gaf.fit(clf,X_train, y_train,X_test, y_test,method=1)

iteration  0   0.02270175438596491  |mean  0.06583947368421053  |best_score  0.02270175438596491
iteration  1   0.021701754385964913  |mean  0.05508245614035088  |best_score  0.021701754385964913
iteration  2   0.021701754385964913  |mean  0.03980263157894737  |best_score  0.021701754385964913
iteration  3   0.022035087719298244  |mean  0.029080701754385964  |best_score  0.021701754385964913
iteration  4   0.021701754385964913  |mean  0.029632456140350878  |best_score  0.021701754385964913
iteration  5   0.021701754385964913  |mean  0.022435087719298245  |best_score  0.021701754385964913
iteration  6   0.021701754385964913  |mean  0.022501754385964908  |best_score  0.021701754385964913
iteration  7   0.021701754385964913  |mean  0.022368421052631576  |best_score  0.021701754385964913
iteration  8   0.021701754385964913  |mean  0.022335087719298242  |best_score  0.021701754385964913
iteration  9   0.021701754385964913  |mean  0.022335087719298242  |best_score  0.021701754385964913
itera